# Classifier 학습
데이터 처리부터 학습까지

## Data preparing
일반적인 이미지, 텍스트, 오디오, 비디오데이터 처리 과정
1. 표준 python 패키지로 불러옴
2. NumPy 배열로 변환
3. torch.Tensor로 변환

유용한 library
- 이미지는 Pillow 나 OpenCV
- 오디오는 SciPy 나 LibROSA
- 텍스트 Python/Cython NLTK SpaCy

torchvision 패키지
- 영상분야를 위한 패키지
- Imagenet/CIFAR10/MNIST 등의 일반적인 데이터셋을 불러오는 함수들 제공
- torchvision.datasets, torch.utils.data.DataLoader

### Classifier 학습하기(MNIST)

1. MNIST 데이터셋 불러오기(torchvision)이용
2. Neural net 정의
3. loss function 및 optimization 정의
4. train data를 이용하여 학습
5. test data를 이용하여 결과 평가

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

In [39]:
#데이터셋 불러오기
# MNIST : 1*28*28 size의 손글씨 dataset 체널 1의 m,std = 0.1307, 0.3081

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.1307,), (0.3081,))])

trainset = torchvision.datasets.MNIST(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.MNIST(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)


In [64]:
# Neural net 정의
# Covolution > relu > pool > Convolution > relu > pool 
# > fc1 > relu > fc2 > relu > fc3

from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

class MNIST_NET(nn.Module):
    def __init__(self):
        super(MNIST_NET, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(6, 16, 3)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16*5*5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
net = MNIST_NET()
print(net)

MNIST_NET(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [65]:
# loss function 및 optimizer 정의
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001)

In [66]:
# neural net 학습하기
total_epoch = 3

for epoch in range(total_epoch):
    #현재 loss 저장
    current_loss = 0.0
    #enumerate : 현제 index와 data를 쌍으로 반환 
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        
        #input으로 사용하기위해 pytorch Variable로 감쌈
        inputs, labels = Variable(inputs), Variable(labels)
        
        #grad 초기화(누적방지)
        optimizer.zero_grad()
        
        #학습, back propagation, optimizing
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        #학습 통계 출력
        current_loss += loss.data[0]
        if i % 1000 == 999:
            print('[Epoch : %d, Iteration : %5d] loss : %.3f' % (epoch+1, i+1, current_loss/1000))
            current_loss = 0.0
print('Finished Training')

/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:24: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


[1,  1000] loss : 2.260
[1,  2000] loss : 2.076
[1,  3000] loss : 1.264
[1,  4000] loss : 0.640
[1,  5000] loss : 0.514
[1,  6000] loss : 0.421
[1,  7000] loss : 0.391
[1,  8000] loss : 0.350
[1,  9000] loss : 0.317
[1, 10000] loss : 0.308
[1, 11000] loss : 0.283
[1, 12000] loss : 0.262
[1, 13000] loss : 0.238
[1, 14000] loss : 0.219
[1, 15000] loss : 0.200
[2,  1000] loss : 0.181
[2,  2000] loss : 0.183
[2,  3000] loss : 0.201
[2,  4000] loss : 0.170
[2,  5000] loss : 0.167
[2,  6000] loss : 0.164
[2,  7000] loss : 0.165
[2,  8000] loss : 0.149
[2,  9000] loss : 0.141
[2, 10000] loss : 0.133
[2, 11000] loss : 0.144
[2, 12000] loss : 0.138
[2, 13000] loss : 0.145
[2, 14000] loss : 0.127
[2, 15000] loss : 0.128
[3,  1000] loss : 0.108
[3,  2000] loss : 0.125
[3,  3000] loss : 0.122
[3,  4000] loss : 0.109
[3,  5000] loss : 0.126
[3,  6000] loss : 0.090
[3,  7000] loss : 0.099
[3,  8000] loss : 0.124
[3,  9000] loss : 0.099
[3, 10000] loss : 0.113
[3, 11000] loss : 0.090
[3, 12000] loss 

In [93]:
#test set에 대해서 결과 확인
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))


for data in testloader:
    images, labels = data
    outputs = net(images)
    #output size = torch.Size([4, 10])
    #torch.max(input, dim, keepdim=False, out=None) -> (Tensor, LongTensor)
    _, predicted = torch.max(outputs.data, 1)
    
    #numpy 배열로 변환 -> 안해주면 error남
    c = (predicted == labels).numpy()
    for i in range(len(c)):
        label = labels[i]
        class_correct[label] += c[i]
        class_total[label] += 1

for i in range(10):
    print('Accuracy of %d : %d %%' % (i, 100*class_correct[i] / class_total[i]))

Accuracy of 0 : 99 %
Accuracy of 1 : 98 %
Accuracy of 2 : 95 %
Accuracy of 3 : 97 %
Accuracy of 4 : 91 %
Accuracy of 5 : 98 %
Accuracy of 6 : 96 %
Accuracy of 7 : 94 %
Accuracy of 8 : 97 %
Accuracy of 9 : 97 %
